In [58]:
import smtplib
import sys

from ultralytics import YOLO
import numpy as np
import os
import matplotlib as mpl
from matplotlib import pyplot as plt
import librosa
from PIL import Image
import io
import cv2
from IPython.display import display
import time
from glob import glob
import random
import datetime
import smtplib
from email.message import EmailMessage


In [79]:
appKey = "dsla fups yjdh qlyf"
 

def send_message(senderEmail, phone_number, carrier, message):
    
    CARRIERS = {
        "att": "@mms.att.net",
        "tmobile": "@tmomail.net",
        "verizon": "@vtext.com",
        "sprint": "@messaging.sprintpcs.com"
    }
    
    if carrier not in CARRIERS:
        raise ValueError("Carrier not supported")

    msg = EmailMessage()
    msg.set_content(message)
    
    recipient = phone_number + CARRIERS[carrier]

    msg['From'] = senderEmail
    msg['To'] = recipient
    msg['Subject'] = ' '
    
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()

    server.login(senderEmail, appKey)
    server.send_message(msg)
    server.quit()
     

In [3]:
# load trained models
fan_model = YOLO('/Users/sayyedjilani/datasci210/full_models/runs/classify/train/weights/last.pt')
pump_model = YOLO('/Users/sayyedjilani/datasci210/full_models/runs/classify/train2/weights/last.pt')
slider_model = YOLO('/Users/sayyedjilani/datasci210/full_models/runs/classify/train3/weights/last.pt')
valve_model = YOLO('/Users/sayyedjilani/datasci210/full_models/runs/classify/train4/weights/last.pt')

In [4]:
def predict_from_audio(audio_path, model):
    
    # Load the .wav file
    audio_vector, sr = librosa.load(audio_path, sr=None)
    
    # Set the figure size and DPI to get a 224x224 image
    fig_size = 224 / 100  # inches (224 pixels / 100 DPI)
    dpi = 100  # Dots per inch

    # Plot and save Mel spectrogram as an image
    fig, ax = plt.subplots(figsize=(fig_size, fig_size), dpi=dpi)
    ax.set_axis_off()
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Remove padding
    
    sgram = librosa.stft(audio_vector)  # Extract short time fourier transform
    sgram_mag, _ = librosa.magphase(sgram)
    mel_scale_sgram = librosa.feature.melspectrogram(S=sgram_mag, sr=sr)
    mel_sgram = librosa.amplitude_to_db(mel_scale_sgram, ref=np.min)  # Decibel scale
    
    librosa.display.specshow(mel_sgram, sr=sr, ax=ax)

    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    plt.close()

    # Collect Mel from memory buffer
    buf.seek(0)
    mel_sgram_image = Image.open(buf).convert('RGB')

    
    # Make prediction on Mel
    pred = model(mel_sgram_image,verbose=False)
    pred_class = np.argmax(pred[0].probs.data.numpy())  

    
    # Output string values for the classification
    if pred_class == 0:
        pred_class = 'abnormal'
    else:
        pred_class = 'normal'
    
    return pred_class



In [84]:
# fan audio stream
fans = glob('/Users/sayyedjilani/datasci210/*_fan/*/normal/*.wav')
random.shuffle(fans)
fans = fans[:10]

# Get the list of abnormal fan audio files and shuffle it
fans_ab = glob('/Users/sayyedjilani/datasci210/*_fan/*/abnormal/*.wav')
random.shuffle(fans_ab)
fans_ab = fans_ab[:3]

fans.extend(fans_ab)
random.shuffle(fans)


In [86]:
fans

['/Users/sayyedjilani/datasci210/6dB_fan/id_02/abnormal/00000007.wav',
 '/Users/sayyedjilani/datasci210/-6dB_fan/id_04/normal/00000489.wav',
 '/Users/sayyedjilani/datasci210/6dB_fan/id_00/normal/00000228.wav',
 '/Users/sayyedjilani/datasci210/6dB_fan/id_06/normal/00000222.wav',
 '/Users/sayyedjilani/datasci210/0dB_fan/id_02/abnormal/00000068.wav',
 '/Users/sayyedjilani/datasci210/-6dB_fan/id_04/normal/00000031.wav',
 '/Users/sayyedjilani/datasci210/6dB_fan/id_02/normal/00000232.wav',
 '/Users/sayyedjilani/datasci210/6dB_fan/id_00/normal/00000930.wav',
 '/Users/sayyedjilani/datasci210/0dB_fan/id_06/normal/00000273.wav',
 '/Users/sayyedjilani/datasci210/6dB_fan/id_06/normal/00000755.wav',
 '/Users/sayyedjilani/datasci210/0dB_fan/id_06/normal/00000156.wav',
 '/Users/sayyedjilani/datasci210/0dB_fan/id_04/abnormal/00000338.wav',
 '/Users/sayyedjilani/datasci210/0dB_fan/id_00/normal/00000860.wav']

In [85]:
# fan audio stream
fans = glob('/Users/sayyedjilani/datasci210/*_fan/*/normal/*.wav')
random.shuffle(fans)
fans = fans[:10]

# Get the list of abnormal fan audio files and shuffle it
fans_ab = glob('/Users/sayyedjilani/datasci210/*_fan/*/abnormal/*.wav')
random.shuffle(fans_ab)
fans_ab = fans_ab[:3]

fans.extend(fans_ab)
random.shuffle(fans)


for fan_audio in fans:
    
    pred = predict_from_audio(fan_audio, fan_model)

    id = fan_audio.split('/')[-3]
    audio = fan_audio.split('/')[-1]

    message = f"""POTENTIAL MECHANICAL FAILURE:
    Machine: FAN
    ID: {id}
    FILE NAME: {audio}
    WHEN: {datetime.datetime.now()}
    
    """

    if pred == 'abnormal':
        send_message('sjilani07@gmail.com','7328615493', 'att', message = message)
        
    time.sleep(10)
        